I add the meaning almost for each cell for starter can easily understand whats the usage. rename some variables name to easy read

# Import some libs

In [1]:
import sys
from pathlib import Path
import subprocess
import os
import gc
from glob import glob

import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


In [2]:
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

# define some functions for load dataset

In [4]:
class Pipeline:

    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
        return df

    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))  #!!?
                df = df.with_columns(pl.col(col).dt.total_days()) # t - t-1
        df = df.drop("date_decision", "MONTH")
        return df

    def filter_cols(df):
        # drop the col if missing value > 0.7
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()
                if isnull > 0.7:
                    df = df.drop(col)
        # Remove Irrelevant or Overly Complex Categorical Columns:
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)
        
        return df

In [5]:
class Aggregator:
    #Please add or subtract features yourself, be aware that too many features will take up too much space.
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        return expr_max +expr_last+expr_mean
    
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]
        return  expr_max +expr_last+expr_mean
    
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        #expr_count = [pl.count(col).alias(f"count_{col}") for col in cols]
        return  expr_max +expr_last#+expr_count
    
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max +expr_last
    
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols] 
        #expr_min = [pl.min(col).alias(f"min_{col}") for col in cols]
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]
        #expr_first = [pl.first(col).alias(f"first_{col}") for col in cols]
        return  expr_max +expr_last
    
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

## how to read file

In [8]:
def read_file(path, depth = None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    if depth in [1,2]:
        df = df.group_by('case_id').agg(Aggregator.get_exprs(df))
    return df

def read_files(regex_path,depth = None):
    chunks = []
    
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = df.group_by('case_id').agg(Aggregator.get_exprs(df))
        chunks.append(df)
        
    df = pl.concat(chunks,how = 'vertical_relaxed')
    df = df.unique(subset = ['case_id'])
    return df

## feature engineering: add new col in df_base & left join different depth file togather

In [9]:
def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base
        .with_columns(
            month_decision = pl.col("date_decision").dt.month(),
            weekday_decision = pl.col("date_decision").dt.weekday(),
        )
    )
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base

In [11]:
def to_pandas(dataframe, categorical_columns=None):
    # Convert the dataframe to a pandas dataframe
    pandas_df = dataframe.to_pandas()
    
    # If categorical_columns is not provided, identify all object dtype columns as categorical
    if categorical_columns is None:
        categorical_columns = list(pandas_df.select_dtypes("object").columns)
    
    # Convert identified columns to categorical dtype
    pandas_df[categorical_columns] = pandas_df[categorical_columns].astype("category")
    
    # Return the pandas dataframe and the list of categorical columns
    return pandas_df, categorical_columns

## decrease the memory by using lightest number type

In [13]:
def reduce_mem_usage(dataframe):
    # Calculate the initial memory usage of the DataFrame
    initial_memory_usage = dataframe.memory_usage().sum() / 1024**2
    print(f'Initial memory usage: {initial_memory_usage:.2f} MB')
    
    # Iterate over each column in the DataFrame
    for column_name in dataframe.columns:
        column_dtype = dataframe[column_name].dtype

        # Skip the optimization for categorical columns
        if str(column_dtype) == "category":
            continue

        # Skip optimization for non-numeric columns
        if column_dtype != 'object':
            min_value = dataframe[column_name].min()
            max_value = dataframe[column_name].max()

            # Downcast integer columns to the smallest integer dtype possible
            if 'int' in str(column_dtype):
                if min_value >= np.iinfo(np.int8).min and max_value <= np.iinfo(np.int8).max:
                    dataframe[column_name] = dataframe[column_name].astype(np.int8)
                elif min_value >= np.iinfo(np.int16).min and max_value <= np.iinfo(np.int16).max:
                    dataframe[column_name] = dataframe[column_name].astype(np.int16)
                elif min_value >= np.iinfo(np.int32).min and max_value <= np.iinfo(np.int32).max:
                    dataframe[column_name] = dataframe[column_name].astype(np.int32)
                elif min_value >= np.iinfo(np.int64).min and max_value <= np.iinfo(np.int64).max:
                    dataframe[column_name] = dataframe[column_name].astype(np.int64)  

            # Downcast float columns to the smallest float dtype possible
            else:
                if min_value >= np.finfo(np.float16).min and max_value <= np.finfo(np.float16).max:
                    dataframe[column_name] = dataframe[column_name].astype(np.float16)
                elif min_value >= np.finfo(np.float32).min and max_value <= np.finfo(np.float32).max:
                    dataframe[column_name] = dataframe[column_name].astype(np.float32)
                else:
                    dataframe[column_name] = dataframe[column_name].astype(np.float64)

        else:
            continue

    # Calculate the final memory usage after optimization
    final_memory_usage = dataframe.memory_usage().sum() / 1024**2
    print(f'Final memory usage: {final_memory_usage:.2f} MB (reduced by {(initial_memory_usage - final_memory_usage) / initial_memory_usage * 100:.1f}%)')
    
    # Return the optimized DataFrame
    return dataframe

# load the train files

## define the path

In [14]:
ROOT            = Path('/kaggle/input/home-credit-credit-risk-model-stability')
TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

## load in data_store

In [15]:
data_store = {
    'df_base': read_file(TRAIN_DIR / 'train_base.parquet'),
    'depth_0':[
        read_file(TRAIN_DIR / 'train_static_cb_0.parquet'),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
     'depth_1': [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),  
     ],
     'depth_2':[
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
        read_file(TRAIN_DIR / "train_applprev_2.parquet", 2),
        read_file(TRAIN_DIR / "train_person_2.parquet", 2)
         
     ]
}

## exec feature engi => get the one overall df

In [16]:
df_train = feature_eng(**data_store)#**传参，不需要提前知道字典中有哪些键，可以更加灵活地传递参数
print("train data shape:\t", df_train.shape)#获得DataFrame维度（行与列）

train data shape:	 (1526659, 861)


## Deleted large objects and free up memory sooner

In [18]:
del data_store
gc.collect()

NameError: name 'data_store' is not defined

## Do the filter for all cols, convert to pandas df & reduce the mem usage of df

In [ ]:
df_train = df_train.pipe(Pipeline.filter_cols)
df_train, cat_cols = to_pandas(df_train)
df_train = reduce_mem_usage(df_train)
print("train data shape:\t", df_train.shape)#优化后

## Get the group numerical Columns by Missing Value Counts 

In [ ]:
nums=df_train.select_dtypes(exclude='category').columns#排除分类变量，选择数值型
from itertools import combinations, permutations
#df_train=df_train[nums]
nans_df = df_train[nums].isna()#判断缺失值
nans_groups={}
for col in nums:
    cur_group = nans_df[col].sum()#缺失值相加
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
del nans_df; x=gc.collect()

# as the result, get the nans_groups
# {
#     0: ["column1", "column2"],  # No missing values in these columns
#     5: ["column3"],             # 5 missing values in column3
#     10: ["column4", "column5"], # 10 missing values in each of these columns
# }

## Define two helper func to handle nans_df

In [ ]:
# selecting columns based on the maximum number of unique values.
def reduce_group(groups):
    selected_columns = []  # This will store the column names selected from each group
    for group in groups:
        max_uniques = 0  # Initialize the maximum number of unique values found
        selected_column = group[0]  # Default to the first column in the group as a fallback
        
        for column in group:
            unique_count = df_train[column].nunique()  # Number of unique values in the column
            if unique_count > max_uniques:
                max_uniques = unique_count
                selected_column = column
        
        selected_columns.append(selected_column)
    
    print('Selected columns based on max uniques:', selected_columns)
    return selected_columns



def group_columns_by_correlation(dataframe, correlation_threshold=0.8):
    # Calculate the correlation matrix for the dataframe
    correlation_matrix = dataframe.corr()

    # Initialize a list to hold all groups of correlated columns
    correlated_groups = []

    # List of columns yet to be grouped
    remaining_columns = list(dataframe.columns)

    while remaining_columns:
        # Pop the first column name off the remaining list to create a new correlation group
        current_column = remaining_columns.pop(0)
        current_group = [current_column]
        correlated_columns = [current_column]  # This will hold all columns that correlate with `current_column`

        # Iterate over the remaining columns and check if the correlation with the current column is above the threshold
        for other_column in remaining_columns:
            if correlation_matrix.loc[current_column, other_column] >= correlation_threshold:
                current_group.append(other_column)
                correlated_columns.append(other_column)

        # Append the group to the list of groups
        correlated_groups.append(current_group)

        # Update remaining_columns by removing the correlated columns
        remaining_columns = [col for col in remaining_columns if col not in correlated_columns]

    return correlated_groups


In [ ]:
df_train = feature_eng(**data_store)#**传参，不需要提前知道字典中有哪些键，可以更加灵活地传递参数
print("train data shape:\t", df_train.shape)#获得DataFrame维度（行与列）
del data_store
gc.collect()#内存回收
df_train = df_train.pipe(Pipeline.filter_cols)
df_train, cat_cols = to_pandas(df_train)
df_train = reduce_mem_usage(df_train)
print("train data shape:\t", df_train.shape)#优化后

nums=df_train.select_dtypes(exclude='category').columns#排除分类变量，选择数值型
from itertools import combinations, permutations
#df_train=df_train[nums]
nans_df = df_train[nums].isna()#判断缺失值
nans_groups={}
for col in nums:
    cur_group = nans_df[col].sum()#缺失值相加
    try:
        nans_groups[cur_group].append(col)
    except:
        nans_groups[cur_group]=[col]
del nans_df; x=gc.collect()

# selecting columns based on the maximum number of unique values.
def reduce_group(groups):
    selected_columns = []  # This will store the column names selected from each group
    for group in groups:
        max_uniques = 0  # Initialize the maximum number of unique values found
        selected_column = group[0]  # Default to the first column in the group as a fallback
        
        for column in group:
            unique_count = df_train[column].nunique()  # Number of unique values in the column
            if unique_count > max_uniques:
                max_uniques = unique_count
                selected_column = column
        
        selected_columns.append(selected_column)
    
    print('Selected columns based on max uniques:', selected_columns)
    return selected_columns



def group_columns_by_correlation(dataframe, correlation_threshold=0.8):
    # Calculate the correlation matrix for the dataframe
    correlation_matrix = dataframe.corr()

    # Initialize a list to hold all groups of correlated columns
    correlated_groups = []

    # List of columns yet to be grouped
    remaining_columns = list(dataframe.columns)

    while remaining_columns:
        # Pop the first column name off the remaining list to create a new correlation group
        current_column = remaining_columns.pop(0)
        current_group = [current_column]
        correlated_columns = [current_column]  # This will hold all columns that correlate with `current_column`

        # Iterate over the remaining columns and check if the correlation with the current column is above the threshold
        for other_column in remaining_columns:
            if correlation_matrix.loc[current_column, other_column] >= correlation_threshold:
                current_group.append(other_column)
                correlated_columns.append(other_column)

        # Append the group to the list of groups
        correlated_groups.append(current_group)

        # Update remaining_columns by removing the correlated columns
        remaining_columns = [col for col in remaining_columns if col not in correlated_columns]

    return correlated_groups

uses=[]
for k,v in nans_groups.items():
    if len(v)>1:
            Vs = nans_groups[k]
            #cross_features=list(combinations(Vs, 2))
            #make_corr(Vs)
            grps= group_columns_by_correlation(df_train[Vs], threshold=0.8)
            use=reduce_group(grps)
            uses=uses+use
            #make_corr(use)
    else:
        uses=uses+v
    print('####### NAN count =',k)
print(uses)
print(len(uses))
uses=uses+list(df_train.select_dtypes(include='category').columns)
print(len(uses))
df_train=df_train[uses]

In [ ]:
sample = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/sample_submission.csv")
device='gpu'
#n_samples=200000
n_est=6000
DRY_RUN = True if sample.shape[0] == 10 else False   
if DRY_RUN:
    device='cpu'
    df_train = df_train.iloc[:50000]
    #n_samples=10000
    n_est = 600
print(device)

# load the test file into data_store

In [ ]:
data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
        read_file(TEST_DIR / "test_applprev_2.parquet", 2),
        read_file(TEST_DIR / "test_person_2.parquet", 2)
    ]
}

In [ ]:
df_test = feature_eng(**data_store)
print("test data shape:\t", df_test.shape)
del data_store
gc.collect()
df_test = df_test.select([col for col in df_train.columns if col != "target"])
print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

df_test, cat_cols = to_pandas(df_test, cat_cols)
df_test = reduce_mem_usage(df_test)

gc.collect()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#遍历键k-值v
for k, v in nans_groups.items():
    if len(v) > 1:
        # 检查列是否存在于DataFrame中
        if all(col in df_train.columns for col in v):
            # 计算相关性矩阵
            correlation_matrix = df_train[v].corr()
            
            # 绘制热力图
            plt.figure(figsize=(10, 8))
            sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
            plt.title(f'Correlation Matrix (NAN count = {k})')
            plt.show()
        else:
            print(f"Columns {v} not found in DataFrame.")

    print('####### NAN count =', k)


In [ ]:
y = df_train ['target']
weeks = df_train['WEEK_NUM']
df_train = df_train.drop(columns = ['target','case_id','WEEK_NUM'])
cv = StratifiedGroupKFold(n_splits=5, shuffle=False)

In [ ]:
df_train[cat_cols] = df_train[cat_cols].astype(str)
df_test[cat_cols] = df_test[cat_cols].astype(str)

In [ ]:
from catboost import CatBoostClassifier, Pool

fitted_models = []
cv_scores = []


for idx_train, idx_valid in cv.split(df_train, y, groups=weeks):#
    X_train, y_train = df_train.iloc[idx_train], y.iloc[idx_train]# 
    X_valid, y_valid = df_train.iloc[idx_valid], y.iloc[idx_valid]
    train_pool = Pool(X_train, y_train,cat_features=cat_cols)
    val_pool = Pool(X_valid, y_valid,cat_features=cat_cols)
    clf = CatBoostClassifier(
    eval_metric='AUC',
    task_type='GPU',
    learning_rate=0.03,
    iterations=n_est)
    clf.fit(train_pool, eval_set=val_pool,verbose=300)
    fitted_models.append(clf)
    y_pred_valid = clf.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores.append(auc_score)
    
print("CV AUC scores: ", cv_scores)
print("Maximum CV AUC score: ", max(cv_scores))

In [ ]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

model = VotingModel(fitted_models)

In [ ]:
feature_importance =fitted_models[2].get_feature_importance(type='PredictionValuesChange')

# 获取特征名称
feature_names = X_train.columns

# 对特征重要性进行排序
sorted_idx = np.argsort(feature_importance)

# 绘制特征重要性图
plt.figure(figsize=(20, 60))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), [feature_names[i] for i in sorted_idx])
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('CatBoost Feature Importance')
plt.show()

In [ ]:
df_test = df_test.drop(columns=["WEEK_NUM"])
df_test = df_test.set_index("case_id")


y_pred = pd.Series(model.predict_proba(df_test)[:, 1], index=df_test.index)
df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred
df_subm.to_csv("submission.csv")
df_subm